&nbsp;

# BONUS - Extraction d'un état réduit par sélection
---

&nbsp;

> #### Pourquoi extraire un état réduit de cette manière ?

Quand on construit notre état réduit à partir de la PCA/EOF, on construit des combinaisons linéaires des variables originales. Les modes (EOF1, EOF2, etc.) n'existent pas physiquement et permettent surtout de maximiser la variance expliquée.

Le défaut, c'est que dans un projet où notre objectif est de capturer la dynamique thermique d'une zone géographique définie, entraîner notre modèle sur des données synthétiques peut devenir un obstacle à l'interprétabiltié causale et physique.

&nbsp;

Dans l'optique de pouvoir comparer avec notre état réduit initial, nous allons réaliser une sélection de *features* (une seconde famille de méthode pour réduire un jeu de donnée). Ces méthodes permettent de conserver les variables à variance élevée (celle qui sont les plus explicatives) ou à variance conditionnelle.

&nbsp;

---

&nbsp;

Implémentons une combinaison simple (SINDy + LASSO) adaptée à une application SciML.



In [4]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

# Load deseasonalized SST data
ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

# Extract SST data and reshape for PCA
sst = ds["analysed_sst"]
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # shape (Nt, Nspace)

dt = 1.0  # time step in days

scaler = StandardScaler()
XScaled = scaler.fit_transform(X)

nComponents = 100
pca = PCA(n_components=nComponents)

PCs = pca.fit_transform(XScaled)     # (T, K)
EOFs = pca.components_.T               # (N, K)
explainedVar = pca.explained_variance_ratio_

topKModes = 30

importance = np.zeros(EOFs.shape[0])

for k in range(topKModes):
    importance += explainedVar[k] * np.abs(EOFs[:, k])

nCandidates = 20
candidateIndices = np.argsort(importance)[-nCandidates:]

dXdt = np.gradient(XScaled, dt, axis=0)

# Sparse regression using Lasso

targetIndex = 0  # variable physique ou composante d’intérêt

XCandidates = XScaled[:, candidateIndices]
yTarget = dXdt[:, targetIndex]

lasso = Lasso(alpha=0.005)
lasso.fit(XCandidates, yTarget)

selectedMask = np.abs(lasso.coef_) > 1e-6
selectedIndices = candidateIndices[selectedMask]

XReduced = X[:, selectedIndices]

dXdtReduced = np.gradient(XReduced, dt, axis=0)

# Save the reduced dataset
dsReduced = xr.Dataset(
    {
        "XReduced": (("time", "space"), XReduced),
        "dXdtReduced": (("time", "space"), dXdtReduced),
    },
    coords={
        "time": ds["time"],
        "space": selectedIndices,
    },
)

dsReduced.to_netcdf("data/processed/sstReducedState2COPERNICUS20102019.nc")




